In [1]:
# -*- coding: UTF-8 -*- 
import onnx
from onnx import helper, checker
from onnx import TensorProto
import re
import argparse
import json,copy
import numpy as np
from data_json import *
from onnx_split import *

In [30]:
origin_model = '/home/sky/Downloads/models/resnet50/resnet50-v2-7.onnx'
#origin_model = '/home/sky/Downloads/models/alexnet/bvlcalexnet-9.onnx'
platform_file = './4platform.txt'
nodes_number = 2

In [32]:
graph = load_onnx(origin_model)
input_map = generate_node_dict(graph.input)  
output_map = generate_node_dict(graph.output)
initializer_map = generate_node_dict(graph.initializer)
value_map = generate_node_dict(graph.value_info)       
node_map = generate_node_dict(graph.node)
with open(platform_file, 'r') as f:         
        platform = f.readlines()
        #print (platform)
        for i in range(len(platform)):
            temp = platform[i].split(':')[0]
            platform[i] = temp.replace('\n','')
#print (platform)
platform_num = len(platform)

platform_dict={} 
######platform 对应id
for i in range(platform_num):
    platform_dict[platform[i]] = i
    
print ("platform dict: ", platform_dict)
node_len = len(list(node_map))
mapping_dict ={}


Check input model Errors:  None
platform dict:  {'nx01': 0, 'nx02': 1, 'nx03': 2, 'nx04': 3}


In [33]:
node_list = []
start_index = 0
end_index = 0

for i in range(nodes_number):  
    
    start_index = end_index
    end_index = min((int)(node_len/nodes_number)*(i+1) , node_len)
    #print (start_index, end_index)
    for j in range(start_index, end_index):
        node_list.append(graph.node[j].name)
    new_node_list = copy.copy(node_list)
    mapping_dict['nx0'+str(i+1)]=new_node_list
    node_list = []
print (mapping_dict)
save_json(mapping_dict,"./"+str(nodes_number)+"mapping.json")

{'nx01': ['resnetv24_batchnorm0_fwd', 'resnetv24_conv0_fwd', 'resnetv24_batchnorm1_fwd', 'resnetv24_relu0_fwd', 'resnetv24_pool0_fwd', 'resnetv24_stage1_batchnorm0_fwd', 'resnetv24_stage1_activation0', 'resnetv24_stage1_conv0_fwd', 'resnetv24_stage1_batchnorm1_fwd', 'resnetv24_stage1_activation1', 'resnetv24_stage1_conv1_fwd', 'resnetv24_stage1_batchnorm2_fwd', 'resnetv24_stage1_activation2', 'resnetv24_stage1_conv2_fwd', 'resnetv24_stage1_conv3_fwd', 'resnetv24_stage1__plus0', 'resnetv24_stage1_batchnorm3_fwd', 'resnetv24_stage1_activation3', 'resnetv24_stage1_conv4_fwd', 'resnetv24_stage1_batchnorm4_fwd', 'resnetv24_stage1_activation4', 'resnetv24_stage1_conv5_fwd', 'resnetv24_stage1_batchnorm5_fwd', 'resnetv24_stage1_activation5', 'resnetv24_stage1_conv6_fwd', 'resnetv24_stage1__plus1', 'resnetv24_stage1_batchnorm6_fwd', 'resnetv24_stage1_activation6', 'resnetv24_stage1_conv7_fwd', 'resnetv24_stage1_batchnorm7_fwd', 'resnetv24_stage1_activation7', 'resnetv24_stage1_conv8_fwd', 'resn